In [ ]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport

from matplotlib.ticker import PercentFormatter
plt.rcParams.update({ "figure.figsize" : (8, 5),"axes.facecolor" : "white", "axes.edgecolor":  "black"})
plt.rcParams["figure.facecolor"]= "w"
pd.plotting.register_matplotlib_converters()
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
df = pd.read_json(
    "https://raw.githubusercontent.com/the-pudding/data/master/pockets/measurementRectangles.json"
)  # to not index on user id..as we got duplicates that we need
print(df.columns)

In [ ]:
df.describe()

## Assumptions -> can become tests

- women jeans are more expensive
- women pockets are smaller
- women jeans that are more expensive have smaller pockets -> you should wear a bag with if ur jeans are expensive

In [ ]:
#transforming categorical columns to categorical type: brand     style menWomen
df["brand"] = df["brand"].astype("category")
df["style"] = df["style"].astype("category")
df["menWomen"] = df["menWomen"].astype("category")

### Assumption 1

In [ ]:
df.price.describe()

In [ ]:
bins = [9, 50, 74, 96, 250]
df['price_binned'] = pd.cut(df['price'], bins)

In [ ]:
# - women jeans are more expensive
# women jeans are not really more expensive
pd.crosstab(df["menWomen"], df.price_binned)

### Assumption 2

In [ ]:
df.pocketArea.describe()

In [ ]:
bins = [4454, 5906, 8619, 10725, 13103]
df['pocket_binned'] = pd.cut(df['pocketArea'], bins)

In [ ]:
pd.crosstab(df["menWomen"], df.pocket_binned, normalize="columns")

### Assumption 3

In [ ]:
pd.crosstab(df["price_binned"], df.pocket_binned, normalize="columns")

# Get women pocket size score for each brand

to get this score we can do a simple approach and a more complex one

### Simple score with values from 0 to 2, the number of women pants that are larger than average in that brand

- colums I would add: is_size_smaller_than_average ( per brand / per dataset )

- method: get_pocket_size_score -> return a score based on if women pocket sizes are higher than average... by brand in relation to the rest maybe

## Data Transformations
### Exercise 1: 

#### Creating a column with data transformation of another column

colums I would add: is_size_smaller_than_average ( per brand / per dataset )

In [ ]:
def is_greater_than_average(series: pd.Series) -> pd.Series :
    avg = series.mean()
    new_series = [0 if x <= avg else 1 for x in series]

    return new_series

In [ ]:
df["size_greater_than_average"] = is_greater_than_average(df["pocketArea"])

In [ ]:
is_greater_than_average(df["pocketArea"]).sum()

## Exercise 2

### Pocket size score

method: get_pocket_size_score -> return a score based on if women pocket sizes are higher than average... by brand in relation to the rest maybe

todo: write test

In [ ]:
# count number of rows with women and "size_greater_than_average" > 0
# input brand, menWomen, size_greater_than_average -> 80 rows
# output brand, score  -> less ~ 40


aggr = df.groupby(by=["brand", "menWomen"],as_index=False)["size_greater_than_average"].sum()

In [ ]:
def get_sum_score_by_brand_and_gender(frame: pd.DataFrame, brand_col="brand", gender_col="menWomen", score_by="size_greater_than_average") -> pd.DataFrame :
    aggr = frame.groupby(by=[brand_col, gender_col],as_index=False)[score_by].sum()

    return aggr

In [ ]:
aggr = get_sum_score_by_brand_and_gender(df, "brand", "menWomen", "size_greater_than_average")
aggr

In [ ]:
aggr[aggr.menWomen == "women"]

In [ ]:
#todo: write the test

# Homework


## a less all or nothing score

we sort the pocket areas and assing to each row the index of the area value ( values should be 0 - dataframe size)
- brand women score = average position (sum / 2)
- brand men score = average position (sum / 2)
- score = women / men

todo: 
- write this as a testable method.. might be more than one
- write test

In [ ]:
df_small = df[["brand", "menWomen", "pocketArea"]]

In [ ]:
df_small = df_small.sort_values("pocketArea").reset_index(drop=True)

In [ ]:
df_small['rank'] = df_small.index + 1

In [ ]:
df_small

In [ ]:
aggr = df_small.groupby(by=["brand", "menWomen"],as_index=False)["rank"].sum()

In [ ]:
aggr